## Pydna Gel Electrophoresis Simulation

In [1]:
import pydna
from Bio.Restriction import BamHI, EcoRV
from pydna.gel import gen_sample, weight_standards, weight_standard_sample, lin_div_Qty, random_Dseqs, Q_

# For convenience
def printQ(quantities):
    for Q in quantities:
        print(Q)

def printN(samples):
    for sample in samples:
        print(Q_([seq.n for seq in sample], 'mol').to('pmol'))

### Simple Reconstruction Experiment (for context)

In [2]:
gb = pydna.Genbank("pg25220@alunos.uminho.pt") # Tell Genbank who you are!

AttributeError: module 'pydna' has no attribute 'Genbank'

In [4]:
# Gene
gene = gb.nucleotide("X06997") # Kluyveromyces lactis LAC12 gene for lactose permease.

In [5]:
# Primers
primer_f, primer_r = pydna.parse(''' >760_KlLAC12_rv (20-mer)
                                     ttaaacagattctgcctctg

                                     >759_KlLAC12_fw (19-mer)
                                     aaatggcagatcattcgag
                                     ''', ds=False)

In [6]:
# PCR
pcr_prod = pydna.pcr(primer_f, primer_r, gene)

In [7]:
# Vector
vector = gb.nucleotide("AJ001614") # pCAPs cloning vector

In [8]:
# Linearized vector
lin_vector = vector.linearize(EcoRV)

In [9]:
# Reconstructed vector (linearized vector + pcr product)
rec_vec = (lin_vector + pcr_prod).looped()

In [10]:
print len(pcr_prod),
print len(rec_vec),
print len(lin_vector)

1766 4894 3128


### Sample for Electrophoresis

List of `pydna` **`Dseqrecord`**s or **`Dseq`**s.

In [11]:
# realistic sample
sample1 = [pcr_prod, rec_vec, lin_vector]

print sample1

[Amplicon(1766), Dseqrecord(o4894), Dseqrecord(-3128)]


#### Default quantities

* Each **`Dseq`** object has a **`mw()`** method that returns its approximate molecular weight (**g/mol**) as determined from its sequence.

* Each **`Dseqrecord`** object has a **`n`** property set to a default value (**mol**).

* Each **`Dseqrecord`** object has a **`m()`** method that returns its mass (**g**) as given by **`Dseqrecord.seq.mw() * Dseqrecord.n`**.

In [12]:
print Q_([seq.n for seq in sample1], 'mol').to('pmol')  # pmol
print Q_([seq.m() for seq in sample1], 'g').to('ng')    # ng

[ 0.05  0.05  0.05] pmol
[  54.55532  151.19348   96.64211] ng


### Random Dseqs

Return a list of pydna Dseqs of given sizes and random sequences.

**`random_Dseqs(sizes)`**

In [14]:
# samples of random sequences
sample2 = random_Dseqs([500, 1000, 5000])
sample3 = random_Dseqs([3000, 1500])

print sample2

[Dseq(-500)
nnnn..nnnn
nnnn..nnnn, Dseq(-1000)
nnnn..nnnn
nnnn..nnnn, Dseq(-5000)
nnnn..nnnn
nnnn..nnnn]


# Gel class

### `Gel(samples, names, percentgel=Q_(1.0, 'g/ml'), electrfield=Q_(5.0, 'V/cm'), ...)`

### `G.run(till_len=0.75, till_time=None, exposure=0.5, ...)`

 - Agarose slab gel electrophoresis.
 - Returns a **`matplotlib.pyplot`** object.

**Notes:**
* Converts **`Dseqs`** into **`Dseqrecords`** with the default quantity (**`n`**).
* Uses **`Dseqrecord.m()`** to get each band's DNA quantity. This quantity is related to the area underneath the signal intensity curve.

In [19]:
# Samples (each sample is loaded to a different well)
samples = [sample1, sample2, sample3]

# Instantiate Gel
G = pydna.Gel(samples)

# Run Electrophoresis
G.run()

In [20]:
# DNA quantity in each band (ng) 
printQ(G.quantities)

[  54.55532  151.19348   96.64211] ng
[  3089.79         6178.79        30890.78999999] ng
[ 18534.79   9267.79] ng


In [21]:
# DNA quantity in each band (pmol) 
printN(G.samples)

[ 0.05  0.05  0.05] pmol
[ 10.  10.  10.] pmol
[ 10.  10.] pmol


### Weight standards

**`weight_standard_sample(key, qty=<Quantity(500, 'nanogram')>)`**

Creates a sample respecting the sizes and mass fractions of the ladder standards stored in the dictionary **`weight_standards`**.

In [22]:
# Weight standards stored
weight_standards.keys()

['Mix_GeneRuler', '1kb+_GeneRuler', 'High_Range_GeneRuler', '1kb_GeneRuler']

In [23]:
ladder = weight_standard_sample('1kb_GeneRuler')  # total = 500 ng
print ladder

[Dseqrecord(-10000), Dseqrecord(-8000), Dseqrecord(-6000), Dseqrecord(-5000), Dseqrecord(-4000), Dseqrecord(-3500), Dseqrecord(-3000), Dseqrecord(-2500), Dseqrecord(-2000), Dseqrecord(-1500), Dseqrecord(-1000), Dseqrecord(-750), Dseqrecord(-500), Dseqrecord(-250)]


In [24]:
print Q_([seq.n for seq in ladder], 'mol').to('pmol')  # pmol
print Q_([seq.m() for seq in ladder], 'g').to('ng')    # ng

[ 0.00485588  0.00606983  0.01888381  0.00971163  0.01213946  0.01387361
  0.03776682  0.01618564  0.02023179  0.02697515  0.09710639  0.0539457
  0.08091165  0.16178193] pmol
[ 30.  30.  70.  30.  30.  30.  70.  25.  25.  25.  60.  25.  25.  25.] ng


In [25]:
samples = [sample1, sample2, sample3, ladder]
G = Gel(samples)
gelpic = G.run()
gelpic.show()

NameError: name 'Gel' is not defined

### Assigning Quantities

**`gen_sample(sizes, quantities)`**

Return list of pydna Dseqrecords of given size and quantity.


#### Direct Assignment

**Note:** If no units are provided, nanograms are assumed.

In [26]:
sizes3 = [3000, 1500]  # bp
qts3 = [100, 100]  # ng
sample3 = gen_sample(sizes3, qts3)

print sample3
print Q_([seq.n for seq in sample3], 'mol').to('pmol')  # pmol
print Q_([seq.m() for seq in sample3], 'g').to('ng')    # ng

[Dseqrecord(-3000), Dseqrecord(-1500)]
[ 0.05395259  0.10790059] pmol
[ 100.  100.] ng


#### Assigning Total Quantity

**Note:** For fragments resulting from a digestion with restriction enzyme procedure, for example, the mass of the fragments is proportional to their size. Lets assume that is the case and that we have a total mass of 200 ng.

In [27]:
sizes2 = Q_([0.5, 1, 5], 'kbp')  # this time with declared units
totalQ2 = Q_(200, 'ng')
sample2 = gen_sample(sizes2, totalQ2)

print sample2
print Q_([seq.n for seq in sample2], 'mol').to('pmol')  # pmol
print Q_([seq.m() for seq in sample2], 'g').to('ng')    # ng
print sum(Q_([seq.m() for seq in sample2], 'g').to('ng'))

[Dseqrecord(-500), Dseqrecord(-1000), Dseqrecord(-5000)]
[ 0.04979178  0.04979815  0.04980324] pmol
[  15.38461538   30.76923077  153.84615385] ng
200.0 ng


In [28]:
samples = [ladder, sample1, sample2, sample3]
G = Gel(samples)
gelpic = G.run()
gelpic.show()

NameError: name 'Gel' is not defined

In [29]:
printQ(G.quantities)

[  54.55532  151.19348   96.64211] ng
[  3089.79         6178.79        30890.78999999] ng
[ 18534.79   9267.79] ng


In [30]:
printN(G.samples)

[ 0.05  0.05  0.05] pmol
[ 10.  10.  10.] pmol
[ 10.  10.] pmol


### Stop Criteria

**`till_len`**
* Fraction of the gel length (measured from the well bottom) to
serve as finish line. Defaults to 0.75.

**`till_time`**
* Time (in hours) at which to stop the electrophoresis. Defaults to None.

**Note:** If both conditions are given the most stringent will be respected.

In [31]:
G = Gel(samples)
gelpic = G.run(till_len=1)
gelpic.show()

NameError: name 'Gel' is not defined

In [32]:
G = Gel(samples)
gelpic = G.run(till_len=0.4)
gelpic.show()

NameError: name 'Gel' is not defined

In [33]:
G = Gel(samples)
gelpic = G.run(till_time=Q_(1,'hr'))
gelpic.show()

NameError: name 'Gel' is not defined

In [34]:
G = Gel(samples)
gelpic = G.run(till_len=0.8, till_time=Q_(1,'hr'))
gelpic.show()

NameError: name 'Gel' is not defined

In [35]:
G = Gel(samples)
gelpic = G.run(till_len=0.8, till_time=Q_(3,'hr'))
gelpic.show()

NameError: name 'Gel' is not defined

### Exposure effect

Fraction of signal saturation (0-1).

If exposure is set to 0 every band's light intensity will be given by a perfect Gaussian curve. Wider bands with lower
amounts of DNA might be hard to see.

The closer to 1 the exposure the higher the saturation, which favors the weaker bands visualization.

If exposure is set to 1 only the weakest band will be a Gaussian, all others will be saturated.

In [36]:
G = Gel(samples)
gelpic = G.run(exposure=0)
gelpic.show()

NameError: name 'Gel' is not defined

In [37]:
G = Gel(samples)
gelpic = G.run(exposure=1)
gelpic.show()

NameError: name 'Gel' is not defined

### Agarose Concentration

**`percentgel`**

* Agarose concentration in the gel.
* Defaults to Q_(1.0, '(g/(100 mL))*100').


In [38]:
G = Gel(samples, percentgel=0.5)
gelpic = G.run()
gelpic.show()

NameError: name 'Gel' is not defined

In [39]:
G = Gel(samples, percentgel=1.3)
gelpic = G.run()
gelpic.show()

NameError: name 'Gel' is not defined

### Electric Field Intensity

**`electrfield`**
* Defaults to Q_(5.0, 'V/cm').

In [40]:
G = Gel(samples, electrfield=0.7)
gelpic = G.run()
gelpic.show()
# Notice the time.

NameError: name 'Gel' is not defined

In [41]:
G = Gel(samples, electrfield=Q_(6.0, 'V/cm'))
gelpic = G.run()
gelpic.show()
# Notice the time.

NameError: name 'Gel' is not defined